# Compare model to baseline

## Setup

In [ ]:
tuning_job = "fm2874nw5inl-LightGB-qF33YMu6Yh"

In [ ]:
!pip install -U scikit-learn==1.0.2 --quiet
!pip install sagemaker-experiments --quiet
!pip install -U seaborn --quiet
!pip install -U lightgbm --quiet

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
features = [
    "CHANNEL_CLASS",
    "DISTRIBUTION_CHANNEL",
    "DIVISION_CODE",
    "SIZE_CODE",
    "SILHOUETTE_SHORT",
    "SALES_ORDER_ITEM_VAS_INDICATOR",
    "VAS_CODE_ZP1",
    "VAS_CODE_SK",
    "VAS_CODE_C20",
    "VAS_CODE_C4X",
    "VAS_CODE_PR",
    "VAS_CODE_C90",
    "VAS_CODE_STD",
    "VAS_CODE_CL1",
    "VAS_CODE_LBC",
    "VAS_CODE_SM",
    "VAS_CODE_CU",
    "VAS_CODE_ES",
    "VAS_CODE_C40",
    "VAS_CODE_CTU",
    "VAS_CODE_CLX",
    "VAS_CODE_SZU",
    "VAS_CODE_REST",
    "VAS_CODE_NONE",
    "SHIPPING_LOCATION_CODE",
    "COUNTRY_CODE",
    "CUSTOMER_ACCOUNT_GROUP_CODE",
    "SALES_ORDER_TYPE",
    "FULL_CASE_QUANTITY",
    "SOSL_TOTAL_QTY",
    "SOI_TOTAL_QUANTITY",
    "SOH_TOTAL_QUANTITY",
]

dtypes = {
    "SALES_ORDER_HEADER_NUMBER": str,
    "SALES_ORDER_ITEM_NUMBER": str,
    "SALES_ORDER_SCHEDULE_LINE_NUMBER": str,
    "CHANNEL_CLASS": str,
    "DISTRIBUTION_CHANNEL": str,
    "DIVISION_CODE": str,
    "PRODUCT_CODE": str,
    "SIZE_CODE": str,
    "SILHOUETTE_SHORT": str,
    "SALES_ORDER_ITEM_VAS_INDICATOR": str,
    "VAS_CODE_ZP1": np.int32,
    "VAS_CODE_SK": np.int32,
    "VAS_CODE_C20": np.int32,
    "VAS_CODE_C4X": np.int32,
    "VAS_CODE_PR": np.int32,
    "VAS_CODE_C90": np.int32,
    "VAS_CODE_STD": np.int32,
    "VAS_CODE_CL1": np.int32,
    "VAS_CODE_LBC": np.int32,
    "VAS_CODE_SM": np.int32,
    "VAS_CODE_CU": np.int32,
    "VAS_CODE_ES": np.int32,
    "VAS_CODE_C40": np.int32,
    "VAS_CODE_CTU": np.int32,
    "VAS_CODE_CLX": np.int32,
    "VAS_CODE_SZU": np.int32,
    "VAS_CODE_REST": np.int32,
    "VAS_CODE_NONE": np.int32,
    "SHIPPING_LOCATION_CODE": str,
    "COUNTRY_CODE": str,
    "CUSTOMER_ACCOUNT_GROUP_CODE": str,
    "SALES_ORDER_TYPE": str,
    "FULL_CASE_QUANTITY": np.float64,
    "SOSL_TOTAL_QTY": np.float64,
    "SOI_TOTAL_QUANTITY": np.float64,
    "SOH_TOTAL_QUANTITY": np.float64,
    "NBR_CARTONS": np.int64,
    "NBR_CARTONS_RATIO": np.float64,
}

## Get baseline data

In [ ]:
df_baseline = pd.read_excel("baseline/baseline.xlsx")
df_baseline.head()

In [ ]:
df_baseline = df_baseline[
    [
        "Sales Order Number",
        "Line Number",
        "Material",
        "Customer PO Number",
        "Confirmed Quantity",
        "Delivered Cartons",
        "Remaining Cartons",
    ]
].sort_values("Sales Order Number")

df_baseline["Total Cartons"] = (
    df_baseline["Delivered Cartons"] + df_baseline["Remaining Cartons"]
)
df_baseline = df_baseline.drop(
    columns=["Confirmed Quantity", "Delivered Cartons", "Remaining Cartons"]
)

In [ ]:
df_baseline = df_baseline.groupby(
    [
        "Sales Order Number",
        "Line Number",
        "Material",
        "Customer PO Number",
    ],
    as_index=False,
).agg({"Total Cartons": "sum"})

df_baseline["Schedule Line"] = (
    df_baseline["Material"].str.split("-", 1, expand=True)[1].astype(int)
)

df_baseline

## Get input for baseline data

In [ ]:
import os

import sagemaker
from sagemaker.s3 import S3Downloader, S3Uploader

project = "shapeshifter"
sm_bucket = sagemaker.session.Session().default_bucket()
input_data_path = os.path.join("s3://", sm_bucket, project, "data/input")

# S3Downloader.download(os.path.join(input_data_path, "data.csv"), "full_data")

df_model = pd.read_csv("full_data/data.csv", dtype=dtypes)

In [ ]:
df_model_bu = df_model

In [ ]:
df_model["SILHOUETTE_SHORT"].unique()

In [ ]:
df_model["SIZE_CODE"].unique()

In [ ]:
df_model.groupby("SALES_ORDER_HEADER_NUMBER", group_keys=False).apply(
    lambda x: x.sample(frac=0.6)
)

## Get best model

If the next cell fails the first time after running this notebook, just restart the kernel and try again.

In [ ]:
import pickle
import tarfile

import joblib
from sagemaker.s3 import S3Downloader

tuning_job = sagemaker.tuner.HyperparameterTuner.attach(tuning_job)
best_model = tuning_job.best_estimator()
S3Downloader.download(best_model.model_data, "lightgbm")

t = tarfile.open("lightgbm/model.tar.gz")
t.extractall("lightgbm")

with open("lightgbm/model.pkl", "rb") as f:
    model = pickle.load(f)

In [ ]:
from sklearn.preprocessing import LabelEncoder

categorical_cols = df_model[features].select_dtypes(include=["object"]).columns

df_model[categorical_cols] = df_model[categorical_cols].astype(str)
le = LabelEncoder()
df_model[categorical_cols] = df_model[categorical_cols].apply(
    lambda col: le.fit_transform(col)
)

pred = model.predict(df_model[features])

df_model["Model Prediction"] = list(pred)

<div class="alert alert-info"> 💡 <strong> Grouped data </strong>

Model data has to be grouped as the baseline data is only available at item level for JD Sports.
</div>

In [ ]:
df_model_grouped = (
    df_model[
        [
            "SALES_ORDER_HEADER_NUMBER",
            "SALES_ORDER_ITEM_NUMBER",
            "Model Prediction",
            "NBR_CARTONS_RATIO",
        ]
    ]
    .groupby(["SALES_ORDER_HEADER_NUMBER", "SALES_ORDER_ITEM_NUMBER"], as_index=False)
    .sum()
)

df_model_grouped["SALES_ORDER_HEADER_NUMBER"] = df_model_grouped[
    "SALES_ORDER_HEADER_NUMBER"
].astype(int)
df_model_grouped["SALES_ORDER_ITEM_NUMBER"] = df_model_grouped[
    "SALES_ORDER_ITEM_NUMBER"
].astype(int)

In [ ]:
df_combined = pd.merge(
    left=df_model_grouped,
    right=df_baseline,
    left_on=["SALES_ORDER_HEADER_NUMBER", "SALES_ORDER_ITEM_NUMBER"],
    right_on=["Sales Order Number", "Line Number"],
    how="inner",
)
df_combined.shape

In [ ]:
print(f'Baseline sum: {sum(df_combined["Total Cartons"])}')
print(f'Model sum: {sum(df_combined["Model Prediction"])}')
print(f'Actuals sum: {sum(df_combined["NBR_CARTONS_RATIO"])}')

## Compare with baseline

In [ ]:
df_combined["diff_baseline"] = (
    df_combined["NBR_CARTONS_RATIO"] - df_combined["Total Cartons"]
)
df_combined["diff_model"] = (
    df_combined["NBR_CARTONS_RATIO"] - df_combined["Model Prediction"]
)

In [ ]:
t1 = df_combined[["diff_baseline"]]
t1 = t1.rename(columns={"diff_baseline": "diff"})
t1["type"] = "baseline"
t2 = df_combined[["diff_model"]]
t2 = t2.rename(columns={"diff_model": "diff"})
t2["type"] = "model"

df_diff = pd.concat([t1, t2], ignore_index=True)

In [ ]:
plot = sns.kdeplot(
    data=df_diff.query("diff > -10 & diff < 10"), x="diff", hue="type", fill=True
)

plot.axvline(0, color="r", alpha=0.3)
plot.axvline(1, linestyle="--", color="r", alpha=0.3)
plot.axvline(-1, linestyle="--", color="r", alpha=0.3)

In [ ]:
from sklearn.metrics import (
    mean_absolute_error,
    mean_absolute_percentage_error,
    mean_squared_error,
    r2_score,
)


def get_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)

    return pd.Series({"mse": mse, "r2": r2, "mae": mae, "mape": mape})

In [ ]:
get_metrics(df_combined["NBR_CARTONS_RATIO"], df_combined["Total Cartons"])

In [ ]:
get_metrics(df_combined["NBR_CARTONS_RATIO"], df_combined["Model Prediction"])

In [ ]:
df_diff["diff_abs"] = abs(df_diff["diff"])

In [ ]:
(
    df_diff[["type", "diff_abs"]]
    .query("diff_abs > 10")
    .groupby("type", as_index=False)
    .agg(
        {
            "diff_abs": ["mean", "std", "count"],
        }
    )
)

In [ ]:
df_diff[["type", "diff_abs"]].query("diff_abs > 10").sort_values(
    "diff_abs", ascending=False
).head(10)

In [ ]:
df_combined.shape[0]

In [ ]:
df_model.shape[0]

In [ ]:
df_baseline.shape[0]